# Logan Clark - College To NFL Trends (WR)

Predicting a players NFL Fantasy Football success by using College Football stats along with Random Forest Regression, and Neural Network Regression. 

**Data**

Evaluation Methodology: The current value metric I am using for each player is

Value = (AVG EP Season 1-3 + AVG PPG Season 1-3)/2 + (#Top-12 Finishes * 2)

My logic behind this is this first part averages a players Expected Points and Actual Points. And then I need the second part because I only have Season 1-3 stats for EP and PPG and how long a player performs at a high level is relevant to Dynasty Fantasy Football.





Source: Pahowdy's Database

https://docs.google.com/spreadsheets/d/1D1D9rXo3BXX867qBju77VjZ9YIoAp-nXYFqSGDQgajU/edit#gid=224755041











In [1]:
!pip install tensorflow-addons

import math
import time

import matplotlib.pyplot as plt
import numpy as np
from numpy.random import seed
import pandas as pd

import tensorflow as tf
import tensorflow_addons as tfa

import torch

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor

from sklearn import metrics

tf.keras.backend.set_floatx('float64')

!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py 

from helper_functions import create_tensorboard_callback, plot_loss_curves

     |████████████████████████████████| 1.1 MB 5.5 MB/s 
--2021-10-20 23:53:52--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-10-20 23:53:52 (83.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
def eval_model(model, X_test, y_test, hist):
    print("eval", model.evaluate(X_test, y_test))
    print("predict", model.predict(X_test[:10]))
    pd.DataFrame(hist.history).plot()
    plt.ylabel("loss")
    plt.xlabel("epochs")

metrics = [tf.metrics.mae, tfa.metrics.RSquare(dtype=tf.float32, y_shape=(1,))]
    

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Data Prep

In [22]:
data = pd.read_csv('/content/gdrive/My Drive/pahowdy_cfb_wr_training.csv')
rookie_data = pd.read_csv('/content/gdrive/My Drive/pahowdy_cfb_wr_rookies.csv')

features = (data.columns)[4:77]

print(features)

print(data.head())
print(rookie_data.head())

data = data.to_numpy()
rookie_data = rookie_data.to_numpy()
rookie_names = rookie_data[:,0]

data = np.delete(data, 0, axis=0)

print(data.shape)
print(rookie_data.shape)



Index(['Conference', 'DR', 'DP', 'Designation in last Year',
       'Age IN DRAFT YEAR (9/1/dy)', 'Conference Drafted Rate',
       'Conference Hit Rate', 'Age Adjusted Hit Rate AVG', 'Pre-Draft',
       'Pre-Draft Percentile', 'Post-Draft', 'Post-Draft Percentile', 'DADA',
       'Cr.Year BO', 'Decimal BOA (30%)', 'Cr.Year BO.1', 'Years Played', 'G',
       'AVG PPG', 'rec', 'YARDS', 'YPR', 'REC/g', 'FINAL MS YARDS RK',
       'College Dominator Rating', 'Per Game College Dominator Rating',
       'YPTpA Dominator', 'Yards Dominator', 'Conference.1',
       'Team (Conference Adjusted)', 'Age', 'First RecYds/TmPatt',
       'Best RecYds/TmPatt', 'Last RecYds/TmPatt', 'AVG RecYds/TmPatt',
       'First RecYds/TmPatt Above Conference AVg',
       'Best RecYds/TmPatt Above Conference Avg',
       'Last  RecYds/TmPatt Above Conference Avg',
       'AVG  RecYds/TmPatt Above Conference Avg',
       'First RecYds/TmPatt Above TM Avg', 'Best RecYds/TmPatt Above TM Avg',
       'Last  RecYds/Tm

In [23]:
def encode_cfb_data(input_data):
  ConferenceDict = {'ACC':0, 'ACC (Atlantic)':1, 'ACC (Coastal)':2, 'American':3, 'American (East)':4, 'American (West)':5,
                  'Big 12':6, 'Big 12 (North)':7, 'Big 12 (South)':8, 'Big East':9, 'Big Ten':10, 'Big Ten (East)':11,
                  'Big Ten (Leaders)':12, 'Big Ten (Legends)':13, 'Big Ten (West)':14, 'CUSA': 15, 'CUSA (East)':16,
                  'CUSA (West)':17, 'Ind':18, 'MAC (East)':19, 'MAC (West)':20, 'MWC':21, 'MWC (Mountain)':22,
                  'MWC (West)':23, 'Pac-10':24, 'Pac-12 (North)':25, 'Pac-12 (South)':26, 'SEC (East)':27,
                  'SEC (West)':28, 'Sun Belt':29, 'Sun Belt (East)':30, 'Sun Belt (West)':31, 'WAC':31}

  DraftRoundDict = {'UDFA':9}

  DraftPositionDict = {'UDFA':260}

  LastYearDesignationDict = {'SR':0, 'JR':1, 'SO':2, 'FR':3}

  for player in input_data:
    player[4] = ConferenceDict[(player[4].replace("\xa0", " "))]
    if not (player[5].isdigit()):
      player[5] = DraftRoundDict[player[5]]
    if not (player[6].isdigit()):
      player[6] = DraftPositionDict[player[6]]
    player[7] = LastYearDesignationDict[player[7]]

  return input_data

In [24]:
# Data Encoding
data = encode_cfb_data(data)

rookie_data = encode_cfb_data(rookie_data)

11
0


In [30]:
# Split data into X and y

X = list()
y = list()

X_rookie = list()

for player in data:
  X.append(player[4:77])
  y.append(player[83])

for player in rookie_data:
  X_rookie.append(player[4:77])

print(X[0])

X = np.array(X).astype(float) 
y = np.array(y).astype(float)

X_rookie = np.array(X_rookie).astype(float)

[11 9 260 0 23 0.08199999999999999 0.11599999999999999 0.01 0.0 0.01 0.44
 0.02 19.0 0.0 20.904 1.96 3 18 5.0 30 481 16.0 1.7 6 0.081 0.162 0.0 0.09
 0.039 0.036000000000000004 -0.6970000000000001 1.31 0.0 0.0 1.77 -0.37
 0.0 0.0 0.036000000000000004 -0.038 0.0 0.0 0.02 -0.8 0.0 0.0 -0.69
 0.027999999999999997 0.179 0.145 0.11699999999999999 0.013000000000000001
 0.08900000000000001 0.07200000000000001 0.057999999999999996 -0.09 -0.06
 -0.016 0.0 0.026000000000000002 0.054000000000000006 0.126 0.222 0.069
 -0.377 0.069 -0.1238 -6.03 -0.162 -0.008 0.0 -2.2 256]


In [33]:
# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

# Baseline Random Forest Model

In [34]:
rfReg = RandomForestRegressor(n_estimators=100).fit(X_train, y_train)
rf_r_squared = rfReg.score(X_test, y_test)
rf_r_squared

0.4201439224088015

In [35]:
#List features by importance

feature_importance = rfReg.feature_importances_

feature_numbered_list = list()
for i in range(len(feature_importance)):
  feature_numbered_list.append((feature_importance[i], features[i]))


sorted_features = sorted(feature_numbered_list, reverse=True)
for i in sorted_features:
  print(i)

(0.15251934401819325, 'Post-Draft')
(0.09893896092241879, 'DP')
(0.08975472954716238, 'Age Adjusted Hit Rate AVG')
(0.03115845264293204, 'Post-Draft Percentile')
(0.028720925750209368, 'Combine Score')
(0.028523135273509845, 'rec')
(0.026266172119443366, 'YPR')
(0.024016380582979367, 'Decimal BOA (30%)')
(0.01928299093937453, 'AVG MS REC Yds.1')
(0.017887716818122665, 'PPG Above conference expectation (Best Year)')
(0.01768969582191435, 'AVG MS REC Yds')
(0.01744634533258546, 'YARDS')
(0.01674942879893664, 'First Dominator.1')
(0.01674680563782652, 'First MS rec Yards.1')
(0.014588211401268128, 'First MS rec Yards')
(0.014560754022206537, 'TeamMate Score (TeamMate Over Expected)')
(0.013456507677001186, 'REC/g')
(0.01268842410354327, 'Conference Drafted Rate')
(0.012655898313488357, 'Conference')
(0.012309129160362428, 'Pre-Draft')
(0.01204115157115204, 'AVG PPG')
(0.011849573661224435, 'last ms rec yards.1')
(0.011788329978453959, 'Conference Hit Rate')
(0.01080608633612598, 'Best Rec

# Baseline Neural Network

In [37]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

# Pre-processing layer (normalization)
data_normalization = Sequential([
                                 preprocessing.Normalization()
], name="data_normalization")

# Setup input shape
input_shape = 73

# Assemble model
inputs = layers.Input(shape=input_shape, name="input_layer")
x = data_normalization(inputs)
x = layers.Dense(100)(x)
x = layers.LeakyReLU()(x)
x = layers.Dense(10)(x)
x = layers.LeakyReLU()(x)
outputs = layers.Dense(1, name="output_layer")(x)
model_0 = tf.keras.Model(inputs, outputs)

model_0.compile(loss=tf.keras.losses.mae,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=metrics)

history_0 = model_0.fit(X_train, y_train, epochs=50, callbacks=[create_tensorboard_callback(dir_name="wr_trends",
                                                                                            experiment_name="model_0")])

Saving TensorBoard log files to: wr_trends/model_0/20211021-002640
Epoch 1/50
25/25 [==============================] - 1s 5ms/step - loss: 11.0363 - mean_absolute_error: 11.0363 - r_square: -12.4918
Epoch 2/50
25/25 [==============================] - 0s 2ms/step - loss: 7.0848 - mean_absolute_error: 7.0848 - r_square: -4.1015
Epoch 3/50
25/25 [==============================] - 0s 3ms/step - loss: 5.0216 - mean_absolute_error: 5.0216 - r_square: -1.6919
Epoch 4/50
25/25 [==============================] - 0s 2ms/step - loss: 4.3344 - mean_absolute_error: 4.3344 - r_square: -0.9541
Epoch 5/50
25/25 [==============================] - 0s 3ms/step - loss: 4.6865 - mean_absolute_error: 4.6865 - r_square: -1.4172
Epoch 6/50
25/25 [==============================] - 0s 2ms/step - loss: 4.1549 - mean_absolute_error: 4.1549 - r_square: -0.7976
Epoch 7/50
25/25 [==============================] - 0s 2ms/step - loss: 3.4963 - mean_absolute_error: 3.4963 - r_square: -0.4561
Epoch 8/50
25/25 [=========

In [38]:
model_0.evaluate(X_test, y_test)

7/7 [==============================] - 0s 2ms/step - loss: 4.6689 - mean_absolute_error: 4.6689 - r_square: -0.5777


[4.668935785488206, 4.668935872752467, -0.5776662826538086]

In [39]:
model_1 = tf.keras.Sequential([
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(10),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(1),
])

model_1.compile(loss=tf.keras.losses.mae,
                        optimizer=tf.keras.optimizers.Adam(),
                        metrics=metrics)

history = model_1.fit(X_train, y_train, epochs=50)

Epoch 1/50
25/25 [==============================] - 1s 4ms/step - loss: 7.2896 - mean_absolute_error: 7.2896 - r_square: -6.1975
Epoch 2/50
25/25 [==============================] - 0s 4ms/step - loss: 2.7041 - mean_absolute_error: 2.7041 - r_square: -0.0374
Epoch 3/50
25/25 [==============================] - 0s 4ms/step - loss: 2.7298 - mean_absolute_error: 2.7298 - r_square: -0.0142
Epoch 4/50
25/25 [==============================] - 0s 4ms/step - loss: 2.6011 - mean_absolute_error: 2.6011 - r_square: 0.0455
Epoch 5/50
25/25 [==============================] - 0s 4ms/step - loss: 2.6637 - mean_absolute_error: 2.6637 - r_square: 0.0504
Epoch 6/50
25/25 [==============================] - 0s 4ms/step - loss: 2.5715 - mean_absolute_error: 2.5715 - r_square: 0.0550
Epoch 7/50
25/25 [==============================] - 0s 4ms/step - loss: 2.7674 - mean_absolute_error: 2.7674 - r_square: 0.0321
Epoch 8/50
25/25 [==============================] - 0s 4ms/step - loss: 2.6869 - mean_absolute_error:

In [40]:
model_1.evaluate(X_test, y_test)

7/7 [==============================] - 0s 3ms/step - loss: 2.2736 - mean_absolute_error: 2.2736 - r_square: 0.3389


[2.273577573348065, 2.2735774799482655, 0.338875412940979]

In [57]:
def normalized_predictions_with_names(predictions):
  pred_sum = sum(predictions)

  normalized_preds = [(float(i)/pred_sum)*1000 for i in predictions]

  names_preds = dict()

  for i in range(len(normalized_preds)):
      if type(normalized_preds[i]) is np.ndarray:
        names_preds[rookie_names[i]] = normalized_preds[i][0]
      else:
        names_preds[rookie_names[i]] = normalized_preds[i]   
          
  sorted_rookies = sorted(names_preds.items(), key=lambda x: x[1], reverse=True)

  # for i in sorted_rookies:
  #     print(i)
  return sorted_rookies

def sort_predictions_with_names(predictions):
  names_preds = dict()

  for i in range(len(predictions)):
      if type(predictions[i]) is np.ndarray:
        names_preds[rookie_names[i]] = predictions[i][0]
      else:
        names_preds[rookie_names[i]] = predictions[i]   
          
  sorted_rookies = sorted(names_preds.items(), key=lambda x: x[1], reverse=True)


  return sorted_rookies

In [58]:
rf_preds = rfReg.predict(X_rookie)
rf_preds_sorted = sort_predictions_with_names(rf_preds)
rf_preds_norm = normalized_predictions_with_names(rf_preds)

for i in rf_preds_sorted:
  print(i)

("Ja'Marr Chase", 15.852750000000006)
('Rashod Bateman', 14.019199999999998)
('Chatarius Atwell', 11.204350000000002)
('Devonta Smith', 11.029600000000002)
('Jaylen Waddle', 10.719450000000004)
('Elijah Moore', 9.964499999999997)
('Rondale Moore', 9.791399999999996)
("D'Wayne Eskridge", 9.762050000000004)
('Terrace Marshall Jr.', 9.3456)
('Kadarius Toney', 9.248949999999999)
('Tylan Wallace', 8.42685)
('Dyami Brown', 7.455)
('Anthony Schwartz', 7.272450000000001)
('Jaelon Darden', 6.600150000000002)
('Amon-Ra St. Brown', 6.263000000000001)
('Amari Rodgers', 6.145300000000003)
('Nico Collins', 5.746099999999999)
('Jonathan Adams Jr.', 5.143200000000004)
('Isaiah McKoy', 4.992200000000002)
('Whop Philyor', 4.880100000000006)
('Blake Proehl', 4.868300000000006)
('Jacob Harris', 4.8017)
('Tamorrion Terry', 4.747700000000002)
('Keylon Stokes', 4.670200000000002)
('Dazz Newsome', 4.638450000000003)
('Kekoa Crawford', 4.632500000000001)
('Ronnie Bell', 4.629450000000004)
('Warren Jackson', 4.

In [59]:
model_1_preds = model_1.predict(X_rookie)
model_1_preds_sorted = sort_predictions_with_names(model_1_preds)
model_1_preds_norm = normalized_predictions_with_names(model_1.predict(X_rookie))

for i in model_1_preds_sorted:
  print(i)

('Devonta Smith', 10.62462807077356)
("Ja'Marr Chase", 9.689012252844162)
('Elijah Moore', 9.359683285754668)
('Rashod Bateman', 9.117796467414392)
('Rondale Moore', 7.747795934882225)
('Jaylen Waddle', 7.724272377828674)
('Chatarius Atwell', 7.588314509556288)
('Kadarius Toney', 6.973896656787304)
("D'Wayne Eskridge", 6.619930680581543)
('Tylan Wallace', 6.1567986206500755)
('Terrace Marshall Jr.', 6.064412771691949)
('Dyami Brown', 5.917279334590164)
('Jaelon Darden', 5.358881941471324)
('Jeremiah Haydel', 5.333512599730581)
('Theo Howard', 5.299534418908042)
('Demetris Robertson', 5.208541950453077)
('McLane Mannix', 5.208094280636876)
('T.J. Vasher', 5.192774917578426)
('Tarik Black', 5.177755414531301)
('Whop Philyor', 5.145412135027683)
('JD Spielman', 5.130437772185295)
('Jhamon Ausbon', 5.121247569259918)
('Trevon Grimes', 5.118725993591047)
('Kekoa Crawford', 5.11342681939499)
('Blake Proehl', 5.111460581265012)
('Jonathan Adams Jr.', 5.10955534242094)
('Keylon Stokes', 5.0854

# Model Comparison

* Similar R^2 values

# Training with Data that does not involve Draft Position to search for "Sleepers"

Theory: Players that's value increases the most when the data does not involve draft position are talented players who were not drafted high meaning they would have a higher % chance to breakout than the average player at their draft position.

In [46]:
# Split data into X and y, while removing features that are draft position or are related

X_sleeper = list()
y_sleeper = list()

X_sleeper_rookie = list()

for player in data:
  temp1 = player[7:14]
  temp2 = player[16:77]
  X_sleeper.append(list(temp1)+list(temp2))
  y_sleeper.append(player[83])

for player in rookie_data:
  temp1 = player[7:14]
  temp2 = player[16:77]
  X_sleeper_rookie.append(list(temp1)+list(temp2))

X_sleeper = np.array(X_sleeper).astype(float) 
y_sleeper = np.array(y_sleeper).astype(float)

X_sleeper_rookie = np.array(X_sleeper_rookie).astype(float)

In [49]:
# Split data into train and test
X_train_sleeper, X_test_sleeper, y_train_sleeper, y_test_sleeper = train_test_split(X_sleeper, y_sleeper, test_size=0.2, random_state=1)

X_train_sleeper = tf.convert_to_tensor(X_train_sleeper)
X_test_sleeper = tf.convert_to_tensor(X_test_sleeper)
y_train_sleeper = tf.convert_to_tensor(y_train_sleeper)
y_test_sleeper = tf.convert_to_tensor(y_test_sleeper)

In [50]:
#Train Random Forest
rf_sleeper = RandomForestRegressor(n_estimators=100).fit(X_train_sleeper, y_train_sleeper)
rf_r_squared_sleeper = rf_sleeper.score(X_test_sleeper, y_test_sleeper)
rf_r_squared_sleeper

0.33574047716795596

In [51]:
#List features by importance

sleeper_features = features[3:]

feature_importance = rf_sleeper.feature_importances_

feature_numbered_list = list()
for i in range(len(feature_importance)):
  feature_numbered_list.append((feature_importance[i], sleeper_features[i]))


sorted_features = sorted(feature_numbered_list, reverse=True)
for i in sorted_features:
  print(i)

(0.29546832398030354, 'Age Adjusted Hit Rate AVG')
(0.030289923890059512, 'G')
(0.029982510251535657, 'last ms rec yards')
(0.02869334512355251, 'Production Score')
(0.027038939569495154, 'rec')
(0.02335250768061331, 'Pre-Draft')
(0.022944215135922326, 'DADA')
(0.022341444604133138, 'Best MS rec Yards')
(0.020086040918471524, 'AVG PPG')
(0.016775018620107853, 'Best MS rec Yards.1')
(0.016300523305739088, 'last ms rec yards.1')
(0.0159480866022221, 'Best Doa')
(0.0157073286035947, 'Cr.Year BO.1')
(0.015303187755553792, 'PPG Above conference expectation (Best Year)')
(0.015004793686920051, 'Last Dominator')
(0.01447399555301103, 'Conference Drafted Rate')
(0.013239495270190766, 'AVG MS REC Yds')
(0.01290309829701944, 'Years Played')
(0.012858909919497653, 'S/EX (MS Yards Over Expectation using a 5 year conference AVG of top producer)')
(0.012783351756950955, 'First MS rec Yards.1')
(0.012762379703919507, 'Conference Hit Rate')
(0.012323619804234007, 'Last  RecYds/TmPatt Above TM Avg')
(0

In [52]:
# Retrain model_1 on new data
model_1_sleeper = tf.keras.Sequential([
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(100),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(10),
                            tf.keras.layers.LeakyReLU(),
                            tf.keras.layers.Dense(1),
])

model_1_sleeper.compile(loss=tf.keras.losses.mae,
                        optimizer=tf.keras.optimizers.Adam(),
                        metrics=metrics)

history = model_1_sleeper.fit(X_train_sleeper, y_train_sleeper, epochs=50)

Epoch 1/50
25/25 [==============================] - 1s 5ms/step - loss: 13.9083 - mean_absolute_error: 13.9083 - r_square: -26.2613
Epoch 2/50
25/25 [==============================] - 0s 5ms/step - loss: 3.2466 - mean_absolute_error: 3.2466 - r_square: -0.3262
Epoch 3/50
25/25 [==============================] - 0s 5ms/step - loss: 3.0627 - mean_absolute_error: 3.0627 - r_square: -0.1661
Epoch 4/50
25/25 [==============================] - 0s 4ms/step - loss: 2.7365 - mean_absolute_error: 2.7365 - r_square: -0.0351
Epoch 5/50
25/25 [==============================] - 0s 4ms/step - loss: 2.7218 - mean_absolute_error: 2.7218 - r_square: -0.0222
Epoch 6/50
25/25 [==============================] - 0s 4ms/step - loss: 2.6522 - mean_absolute_error: 2.6522 - r_square: -0.0372
Epoch 7/50
25/25 [==============================] - 0s 5ms/step - loss: 2.7749 - mean_absolute_error: 2.7749 - r_square: -0.0767
Epoch 8/50
25/25 [==============================] - 0s 4ms/step - loss: 2.6901 - mean_absolute

In [53]:
model_1_sleeper.evaluate(X_test_sleeper, y_test_sleeper)

7/7 [==============================] - 0s 3ms/step - loss: 2.6797 - mean_absolute_error: 2.6797 - r_square: 0.0981


[2.679745720357311, 2.6797456775698474, 0.09811854362487793]

In [54]:
rf_preds_sleeper = rf_sleeper.predict(X_sleeper_rookie)
rf_preds_norm_sleeper = normalized_predictions_with_names(rf_sleeper.predict(X_sleeper_rookie))

for i in rf_preds_norm_sleeper:
  print(i)


('Rashod Bateman', 46.50103302619671)
("Ja'Marr Chase", 41.375020628638)
('Chatarius Atwell', 40.42870859359897)
('Elijah Moore', 39.44180033220467)
('Rondale Moore', 35.74147763109041)
('Devonta Smith', 31.494116891312082)
("D'Wayne Eskridge", 29.60227052671971)
('Kadarius Toney', 29.41810986769829)
('Terrace Marshall Jr.', 27.14176591095987)
('Dyami Brown', 20.973628162519912)
('Anthony Schwartz', 19.83537841360213)
('Tylan Wallace', 19.327070108127483)
('Jaylen Waddle', 18.643000362877373)
('Ronnie Bell', 18.433797587215395)
('Jaelon Darden', 17.23519789260477)
('Jhamon Ausbon', 17.09349995310436)
('Jeremiah Holloman', 16.963934219181358)
('Amon-Ra St. Brown', 16.79983836169184)
('Nico Collins', 16.74539897769058)
('Blake Proehl', 16.061173691343317)
('Tamorrion Terry', 15.97640379339849)
('Kekoa Crawford', 15.744025394261687)
('Amari Rodgers', 15.672787571768602)
('Warren Jackson', 15.441497960311818)
('Whop Philyor', 15.391258185933518)
('Sage Surratt', 15.37865935706465)
('Dax Mi

In [55]:
model_1_preds_sleeper = model_1_sleeper.predict(X_sleeper_rookie)
model_1_preds_norm_sleeper = normalized_predictions_with_names(model_1_sleeper.predict(X_sleeper_rookie))

for i in model_1_preds_norm_sleeper:
  print(i)

('Devonta Smith', 24.612917932223603)
('Tylan Wallace', 23.02385694703987)
('Jaelon Darden', 20.537510857013864)
('Elijah Moore', 20.531525150196344)
("Ja'Marr Chase", 20.233923266763423)
('JD Spielman', 20.1227532219992)
('Rashod Bateman', 19.906467146963713)
('Dez Fitzpatrick', 19.77780092346677)
('Simi Fehoko', 19.72616591713696)
('Rondale Moore', 19.52346042015158)
('Chatarius Atwell', 19.415994573816032)
('Jalen Camp', 19.413899150656917)
('Marquez Stevenson', 19.236242225179907)
('Tamorrion Terry', 18.918233980017327)
('Jonathan Adams Jr.', 18.84038486113851)
('Keylon Stokes', 18.82857224036983)
("D'Wayne Eskridge", 18.757959095055142)
('Amon-Ra St. Brown', 18.735103804091363)
('Seth Williams', 18.69591499789076)
('Dyami Brown', 18.665783254924605)
('Dazz Newsome', 18.592165457503356)
('Jaylen Waddle', 18.413877896749653)
('Amari Rodgers', 18.331602801032705)
('Shi Smith', 18.30410302243719)
('Sage Surratt', 18.250615485426547)
('Jahan Dotson', 18.15795258563475)
('Jhamon Ausbon'

In [56]:
# Create list based off differences

diff_list = list()

for i in range(len(rf_preds)):
  diff_list.append(model_1_preds_sleeper[i]-model_1_preds[i])

diff_dict = dict()

for i in range(len(diff_list)):
    if type(diff_list[i]) is np.ndarray:
      diff_dict[rookie_names[i]] = diff_list[i][0]
    else:
      diff_dict[rookie_names[i]] = diff_list[i]   
        
sorted_diff = sorted(diff_dict.items(), key=lambda x: x[1], reverse=True)

for i in sorted_diff:
  print(i)

('Simi Fehoko', 1.9001708491007347)
('Jacob Harris', 1.8236856670927022)
('Jalen Camp', 1.5990708297411338)
('Marquez Stevenson', 1.3990699239164828)
('Anthony Schwartz', 1.3462461004210864)
('Dez Fitzpatrick', 1.229822223545419)
('Shi Smith', 1.1734173882792103)
('Seth Williams', 1.1688499102096594)
('Dazz Newsome', 1.0626854575739673)
('JD Spielman', 1.051669731648988)
('Jaelon Darden', 0.950647305273046)
('Ihmir Smith-Marsette', 0.9399765033596421)
('Tylan Wallace', 0.9165852860756445)
('Amon-Ra St. Brown', 0.8911376424908433)
('Nico Collins', 0.8350009333023216)
('Tamorrion Terry', 0.807577810936353)
('Sage Surratt', 0.7362651113109999)
('Keylon Stokes', 0.6990263019919185)
('Jonathan Adams Jr.', 0.6785832557225673)
('Tyler Vaughns', 0.6631879612917961)
('Jahan Dotson', 0.6620583299994891)
('Amari Rodgers', 0.6582016558718928)
('Dax Milne', 0.6080064845330009)
('Josh Palmer', 0.5703720498698281)
('Isaiah McKoy', 0.535005637238795)
('Jhamon Ausbon', 0.4488698546145038)
('Damon Hazel